In [1]:
!sudo -H pip install -q transformers --upgrade

In [6]:
!sudo -H pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp38-cp38-linux_x86_64.whl

In [1]:
import pandas

In [2]:
import numpy

In [1]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

ImportError: libcudart.so.11.0: cannot open shared object file: No such file or directory

In [1]:
!pip list | grep cuda

nvidia-cuda-cupti-cu11   11.7.101  
nvidia-cuda-nvrtc-cu11   11.7.99   
nvidia-cuda-runtime-cu11 11.7.99   


In [2]:
import pandas

In [4]:
import torch
torch.__version__

'2.0.1+cu117'

In [3]:
model_path = '/data/quantization-trials/merged-model'

In [4]:
quant_path = '/data/quantization-trials/quant-model'

In [7]:
quant_config = {"zero_point":True,
               "q_group_size": 512,
               "w_bit": 4,
               "version": "GEMM"
               }

In [28]:
#load model
# model = AutoAWQForCausalLM.from_pretrained(model_path, **{"low_cpu_mem_usage":True})
model = AutoAWQForCausalLM.from_pretrained(model_path, safetensors=True, device_map='auto')

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [10]:
#Quantize
# dont run it again
model.quantize(tokenizer, quant_config=quant_config)

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

AWQ: 100%|██████████| 24/24 [06:42<00:00, 16.76s/it]


In [13]:
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

('/data/quantization-trials/quant-model/tokenizer_config.json',
 '/data/quantization-trials/quant-model/special_tokens_map.json',
 '/data/quantization-trials/quant-model/vocab.json',
 '/data/quantization-trials/quant-model/merges.txt',
 '/data/quantization-trials/quant-model/added_tokens.json',
 '/data/quantization-trials/quant-model/tokenizer.json')

### Inference on quantized model

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = quant_path
quant_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0")

ImportError: libcudart.so.11.0: cannot open shared object file: No such file or directory

In [ ]:
def predict_from_quant(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = quant_model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [25]:
import time

In [45]:
# Using quant model
start = time.time()
output1 = predict_from_quant("what is science")
print("time taken is :", time.time()-start)

time taken is : 16.735528707504272


In [46]:
print(output1)

</s>what is science?

Science is a method of understanding the world and the world around us. It is a way of thinking about the world and the world around us. It is a way of understanding the world and the world around us.

What is the world?

The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in 

### Inference on un-quantized model

In [35]:
model_id = model_path
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0")

def predict_from_normal(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [47]:
# Using unquant model
start = time.time()
output2 = predict_from_normal("what is science")
print("time taken is :", time.time()-start)

time taken is : 23.660590171813965


In [48]:
print(output2)

</s>what is science?

The word "science" is used to describe a variety of fields of study, including:

biology

chemistry

biology of the brain

biology of the immune system

biology of the nervous system

biology of the reproductive system

biology of the immune system

biology of the nervous system

biology of the reproductive system

biology of the immune system

biology of the reproductive system

biology of the nervous system

biology of the reproductive system

biology of the immune system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive sy

In [36]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_layer_norm): LayerN

In [33]:
quant_model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (v_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (q_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (out_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): WQLinear_GEMM(in_features=2048, out